* Firstly, let's add WaveBeans dependencies as well as logging library.
* Secondly, add all imports that might be useful.

In [1]:
@file:Repository("file:///home/jovyan/maven-local/repository")
@file:Repository("https://dl.bintray.com/wavebeans/wavebeans")
@file:DependsOn("io.wavebeans:lib:0.0.3-SNAPSHOT11")
@file:DependsOn("io.wavebeans:exe:0.0.3-SNAPSHOT11")
@file:DependsOn("io.wavebeans:http:0.0.3-SNAPSHOT11")
@file:DependsOn("ch.qos.logback:logback-classic:1.2.3")
@file:DependsOn("ch.qos.logback:logback-core:1.2.3")

import io.wavebeans.lib.*
import io.wavebeans.lib.io.*
import io.wavebeans.lib.math.*
import io.wavebeans.lib.stream.*
import io.wavebeans.lib.stream.fft.*
import io.wavebeans.lib.stream.window.*
import io.wavebeans.lib.table.*
import io.wavebeans.http.*
import io.wavebeans.execution.*
import java.util.concurrent.TimeUnit.*

val windowSize = 801
val stepSize = 256
val fftSize = 1024

* Start the HTTP Service on port 6800, so we could query all data in a different Jupyter notebook

In [2]:
HttpService(serverPort = 6800).start()

16:50:23.898 [main] DEBUG io.netty.util.internal.logging.InternalLoggerFactory - Using SLF4J as the default logging framework
16:50:23.928 [main] DEBUG io.netty.util.internal.PlatformDependent0 - -Dio.netty.noUnsafe: false
16:50:23.928 [main] DEBUG io.netty.util.internal.PlatformDependent0 - Java version: 8
16:50:23.932 [main] DEBUG io.netty.util.internal.PlatformDependent0 - sun.misc.Unsafe.theUnsafe: available
16:50:23.934 [main] DEBUG io.netty.util.internal.PlatformDependent0 - sun.misc.Unsafe.copyMemory: available
16:50:23.935 [main] DEBUG io.netty.util.internal.PlatformDependent0 - java.nio.Buffer.address: available
16:50:23.937 [main] DEBUG io.netty.util.internal.PlatformDependent0 - direct buffer constructor: available
16:50:23.938 [main] DEBUG io.netty.util.internal.PlatformDependent0 - java.nio.Bits.unaligned: available, true
16:50:23.938 [main] DEBUG io.netty.util.internal.PlatformDependent0 - jdk.internal.misc.Unsafe.allocateUninitializedArray(int): unavailable prior to Java

io.wavebeans.http.HttpService@660abca

* Define our signal processing stream:
    * it is infinite;
    * we store only last 2 minutes in the table.

In [3]:
val triangularWindow = input { sampleOf(1.0) }
    .window(windowSize)
    .triangular()
    .toTable("triangular", 1.s)
                
val hammingWindow = input { sampleOf(1.0) }
    .window(windowSize)
    .hamming()
    .toTable("hamming", 1.s)

16:50:24.739 [main] DEBUG io.wavebeans.lib.WaveBeansClassLoader - Setting new class loader org.jetbrains.kotlin.scripting.compiler.plugin.impl.CompiledScriptClassLoader@c7e027b from:
io.wavebeans.lib.WaveBeansClassLoader$addClassLoader$1.invoke(WaveBeansClassLoader.kt:24)
io.wavebeans.lib.WaveBeansClassLoader$addClassLoader$1.invoke(WaveBeansClassLoader.kt:5)
mu.internal.LocationAwareKLogger.debug(LocationAwareKLogger.kt:624)
io.wavebeans.lib.WaveBeansClassLoader.addClassLoader(WaveBeansClassLoader.kt:22)
io.wavebeans.lib.Fn$Companion.wrap(Fn.kt:63)
io.wavebeans.lib.io.FunctionInputKt.input(FunctionInput.kt:7)
Line_3_jupyter.<init>(Unknown Source)
sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.lang.reflect.Constructor.newInstance(Constructor.java:423)
kotlin.script.ex

In [4]:
val signal = (880.sine() + 440.sine() + 220.sine())
    .window(windowSize, step = stepSize)

val triangularFft = signal
    .triangular()
    .fft(fftSize)
    .toTable("fft-triangular", 2.m)
val hammingFft = signal
    .hamming()
    .fft(fftSize)
    .toTable("fft-hamming", 2.m)

* Execute the stream, it'll never finish though

In [ ]:
val overseer = LocalDistributedOverseer(
    listOf(triangularFft, hammingFft, triangularWindow, hammingWindow), 
    threadsCount = 2,
    partitionsCount = 1
)

val errors = overseer.eval(44100.0f)
    .map { it.get().exception }
    .filterNotNull()
    .toList()
    
overseer.close()

if (errors.isEmpty()) {
    println("SUCCESS")
} else {
    println("ERROR:\n" + errors.joinToString("\n"))
}
